In [ ]:
from zipfile import ZipFile
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models


In [ ]:
with ZipFile('/content/drive/MyDrive/Colab Notebooks/kagglecatsanddogs_5340.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

In [ ]:
train_dir = '/content/dataset/training_set'
val_dir = '/content/dataset/validation_set'

# Geradores de imagens
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normaliza os valores dos pixels
    rotation_range=30,         # Aumento de dados: rotação
    width_shift_range=0.2,     # Deslocamento horizontal
    height_shift_range=0.2,    # Deslocamento vertical
    shear_range=0.2,           # Inclinação
    zoom_range=0.2,            # Zoom
    horizontal_flip=True       # Inversão horizontal
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Geradores
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),    # Dimensão esperada do InceptionV3
    batch_size=32,
    class_mode='binary'        # Gatos e cães (duas classes)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 25000 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.


In [ ]:
base_model = InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,        # Remove as camadas superiores
    weights='imagenet'        # Pesos pré-treinados no ImageNet
)

# Congelar as camadas do modelo base
base_model.trainable = False

# Construir o modelo
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Reduz as dimensões dos mapas de características
    layers.Dense(128, activation='relu'),  # Camada densa para aprendizado
    layers.Dropout(0.5),                  # Dropout para evitar overfitting
    layers.Dense(1, activation='sigmoid') # Camada de saída binária
])

# Compilar o modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Resumo do modelo
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 8, 8, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,065,185 (84.17 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# Treinamento
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,                  # Número de épocas
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Acurácia na validação: {accuracy:.2f}")


In [ ]:
model.save('inceptionv3_transfer_learning.h5')
